In [1]:
from spamdfba import toolkit as tk
from spamdfba import toymodels as tm
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import pickle
import os
# import seaborn  as sns
# import matplotlib.pyplot as plt
import warnings
import rich
import multiprocessing as mp

2023-10-23 14:26:20,013	INFO worker.py:1642 -- Started a local Ray instance.


In [3]:
NUM_CORES = 8
warnings.filterwarnings("ignore")


## Toy-Exoenzyme-Single-Agent

This toy community is designed to emulate a case that microbial cells are grown on a mixture of starch and glucose in a chemostat with a very low dilution rate of 0.0001, practically a batch system, Figure 3A-C, The strains are capable of secreting amylase to degrade the available starch. However, producing amylase is an energy-consuming step in the organism’s metabolism and it requires ATP that would otherwise be used in biomass production. 

First, we need to define the agents. The agents need a metabolic model (cobra model) which is defined in Toy_Model.py. The agents
also need a name, a neural network class, not instance as a pytorch module, clip which shows the threshold for clipping the gradients, and a learning rate. you also need to define what environment states you want your agent to sense and also what is the actions that the agents can take. Look below for an example of defining an agent.
additionally, you can look at toolkit.py for more information on defining agents.




In [4]:
agent1=tk.Agent("agent1",
                model=tm.ToyModel_SA.copy(),
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=4,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1' ,'Glc', 'Starch'],
                actions=["Amylase_e"],
                gamma=1,
                tau=0.1
                )

agents=[agent1]


After defining the agents, we need to define the environment. The environment needs a list of agents, inictial conditions and a dictionary representing extracellular reactions as well as the duration of an episode and the time resoloution of the DFBA algorithm. More information on defining the environment can be found in toolkit.py. Look below for an example of defining the environment.

In [5]:
env=tk.Environment(name="Toy-Exoenzyme-Single-agents",
                    agents=agents,
                    dilution_rate=0.0001,
                    initial_condition={"Glc":100,"agent1":0.1,"Starch":10},
                    inlet_conditions={"Starch":10},
                    extracellular_reactions=[{"reaction":{
                    "Glc":10,
                    "Starch":-0.1,},
                    "kinetics": (tk.general_kinetic,("Glc","Amylase"))}],
                     dt=0.1,
                     number_of_batches=50,
                     episodes_per_batch=int(NUM_CORES/2),)

The following species are not in the community: ['Starch']


Now we can start the training loop. The training loop needs the environment, the number of episodes, the number of steps per episode, and the number of steps for each gradient update. The training loop will return the rewards for each episode and the average reward for each episode. Look below for an example of training loop.

In [6]:
sim=tk.Simulation(name="Starch_amylase_single",
                  env=env,
                  save_dir="./Results/",
                  )

In [7]:
sim.run()

Hold on, bringing the creitc network to range ...
Done!
Batch 0 finished:
agent1 return was:  -691.9799259050887
Batch 1 finished:
agent1 return was:  -694.6596566901848
Batch 2 finished:
agent1 return was:  -691.0036996004408
Batch 3 finished:
agent1 return was:  -698.9103302819636
Batch 4 finished:
agent1 return was:  -693.7709285641647
Batch 5 finished:
agent1 return was:  -696.4761105785169
Batch 6 finished:
agent1 return was:  -702.5075773106412
Batch 7 finished:
agent1 return was:  -696.4718172243971
Batch 8 finished:
agent1 return was:  -694.4911753806011
Batch 9 finished:
agent1 return was:  -691.3398926101797
Batch 10 finished:
agent1 return was:  -684.5602857094934
Batch 11 finished:
agent1 return was:  -691.4580766812104
Batch 12 finished:
agent1 return was:  -691.2026672651191
Batch 13 finished:
agent1 return was:  -683.5813871723199
Batch 14 finished:
agent1 return was:  -682.359433543093
Batch 15 finished:
agent1 return was:  -685.525736763581
Batch 16 finished:
agent1 re

In [8]:
fig=sim.plot_learning_curves()

In [8]:
sim.print_training_times()

                        Simulation times                         
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Level        ┃ Mean                   ┃ STD                   ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩
│ Optimization │ 0.0002770591378211975  │ 4.193920807812129e-06 │
│ Step         │ 0.00039991034865379335 │ 4.628349008811643e-06 │
│ Batch        │ 1.1968899726867677     │ 0.793404399593946     │
│ Simulation   │ 9.045735836029053      │ NA                    │
└──────────────┴────────────────────────┴───────────────────────┘

In [11]:
agent1=tk.Agent("agent1",
                model=tm.ToyModel_SA.copy(),
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=10,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1','agent2' ,'Glc', 'Starch'],
                actions=["Amylase_e"],
                gamma=1,
                tau=0.1
                )

agent2=tk.Agent("agent2",
                model=tm.ToyModel_SA.copy(),
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=10,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1','agent2', 'Glc', 'Starch'],
                actions=["Amylase_e"],
                gamma=1,
                tau=0.1
                )

agents=[agent1,agent2]

env=tk.Environment(name="Toy-Exoenzyme-Two-agents",
                    agents=agents,
                    dilution_rate=0.0001,
                    initial_condition={"Glc":100,"agent1":0.1,"agent2":0.1,"Starch":10},
                    inlet_conditions={"Starch":10},
                    extracellular_reactions=[{"reaction":{
                    "Glc":10,
                    "Starch":-0.1,},
                    "kinetics": (tk.general_kinetic,("Glc","Amylase"))}],
                    max_c={'Glc':100,
                           'agent1':10,  
                           'Starch':10,
                           },
                           dt=0.1,
                           episode_time=100,
                           number_of_batches=5000,
                           episodes_per_batch=int(NUM_CORES/2),
                           )



Read LP format model from file /var/folders/jk/fr50qn391k794svhntbw333c0000gn/T/tmpalrbsvhd.lp
Reading time = 0.00 seconds
: 7 rows, 18 columns, 44 nonzeros
Read LP format model from file /var/folders/jk/fr50qn391k794svhntbw333c0000gn/T/tmp566lcqnu.lp
Reading time = 0.00 seconds
: 7 rows, 18 columns, 44 nonzeros


The following species are not in the community: ['Starch']


Environment Toy-Exoenzyme-Two-agents created successfully!.


In [13]:
env.rewards={agent.name:[] for agent in env.agents}

if not os.path.exists(f"Results/{env.name}"):
	os.makedirs(f"Results/{env.name}")

for agent in env.agents:
	agent.model.solver="glpk"

for batch in range(env.number_of_batches):

	batch_obs,batch_acts, batch_log_probs, batch_rtgs=tk.rollout(env)
	for agent in env.agents:
		V, _= agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
		A_k = batch_rtgs[agent.name] - V.detach()   
		A_k = (A_k - A_k.mean()) / (A_k.std() + 1e-5) 
		if batch==0:
			rich.print("[bold yellow] Hold on, bringing the creitc network to range...[/bold yellow]")
			err=21
			while err>20:
				V, _= agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
				critic_loss = nn.MSELoss()(V, batch_rtgs[agent.name])
				agent.optimizer_value_.zero_grad()
				critic_loss.backward()
				agent.optimizer_value_.step() 
				err=critic_loss.item()
			rich.print("[bold green] Done![/bold green]")
		else: 
			
			for _ in range(agent.grad_updates):                                                      
				V, curr_log_probs = agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
				ratios = torch.exp(curr_log_probs - batch_log_probs[agent.name])
				surr1 = ratios * A_k.detach()
				surr2 = torch.clamp(ratios, 1 - agent.clip, 1 + agent.clip) * A_k
				actor_loss = (-torch.min(surr1, surr2)).mean()
				critic_loss = nn.MSELoss()(V, batch_rtgs[agent.name])
				agent.optimizer_policy_.zero_grad()
				actor_loss.backward(retain_graph=False)
				agent.optimizer_policy_.step()
				agent.optimizer_value_.zero_grad()
				critic_loss.backward()
				agent.optimizer_value_.step()                                                            
	
		if batch%500==0:
		
			with open(f"Results/{env.name}/{env.name}_{batch}.pkl", 'wb') as f:
				pickle.dump(env, f)
			with open(f"Results/{env.name}/observations_{batch}.pkl", 'wb') as f:
				pickle.dump(batch_obs,f)		
			with open(f"Results/{env.name}/actions_{batch}.pkl", 'wb') as f:
				pickle.dump(batch_acts,f)		
          		
		print(f"Batch {batch} finished:")
		for agent in env.agents:
			print(f"{agent.name} return is:  {np.mean(env.rewards[agent.name][-env.episodes_per_batch:])}")	

 Hold on, bringing the creitc network to range...

 Done!

Batch 0 finished:
agent1 return is:  -834.9255180536776
agent2 return is:  -815.9404837891703


 Hold on, bringing the creitc network to range...

 Done!

Batch 0 finished:
agent1 return is:  -834.9255180536776
agent2 return is:  -815.9404837891703
Batch 1 finished:
agent1 return is:  -841.8440040666383
agent2 return is:  -826.1826531004805
Batch 1 finished:
agent1 return is:  -841.8440040666383
agent2 return is:  -826.1826531004805
Batch 2 finished:
agent1 return is:  -835.6765195038785
agent2 return is:  -820.6359854544567
Batch 2 finished:
agent1 return is:  -835.6765195038785
agent2 return is:  -820.6359854544567
Batch 3 finished:
agent1 return is:  -838.8587132565848
agent2 return is:  -824.0484213947282
Batch 3 finished:
agent1 return is:  -838.8587132565848
agent2 return is:  -824.0484213947282
Batch 4 finished:
agent1 return is:  -841.3364360564342
agent2 return is:  -832.9258086808351
Batch 4 finished:
agent1 return is:  -841.3364360564342
agent2 return is:  -832.9258086808351


KeyboardInterrupt: 

In [ ]:
agent1=tk.Agent("agent1",
                model=tm.ToyModel_SA.copy(),
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=10,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1','agent2','agent3','agent4','agent5','Glc', 'Starch'],
                actions=["Amylase_e"],
                gamma=1,
                tau=0.1
                )

agent2=tk.Agent("agent2",
                model=tm.ToyModel_SA.copy(),
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=10,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1','agent2','agent3','agent4','agent5', 'Glc', 'Starch'],
                actions=["Amylase_e"],
                gamma=1,
                tau=0.1
                )

agent3=tk.Agent("agent3",
                model=tm.ToyModel_SA.copy(),
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=10,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1','agent2','agent3','agent4','agent5', 'Glc', 'Starch'],
                actions=["Amylase_e"],
                gamma=1,
                tau=0.1
                )
agent4=tk.Agent("agent4",
                model=tm.ToyModel_SA.copy(),
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=10,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1','agent2','agent3','agent4','agent5', 'Glc', 'Starch'],
                actions=["Amylase_e"],
                gamma=1,
                tau=0.1
                )
agent5=tk.Agent("agent5",
                model=tm.ToyModel_SA.copy(),
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=10,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1','agent2','agent3','agent4','agent5', 'Glc', 'Starch'],
                actions=["Amylase_e"],
                gamma=1,
                tau=0.1
                )

agents=[agent1,agent2,agent3,agent4,agent5]


env=tk.Environment(name="Toy-Exoenzyme-Five-agents",
                  agents=agents,
                  dilution_rate=0.0001,
                  initial_condition={"Glc":100,"agent1":0.1,"agent2":0.1,"agent3":0.1,"agent4":0.1,"agent5":0.1,"Starch":10},
                  inlet_conditions={"Starch":10},
                  extracellular_reactions=[{"reaction":{
                      "Glc":10,
                  "Starch":-0.1,},
                  "kinetics": (tk.general_kinetic,("Glc","Amylase"))}],
                  max_c={'Glc':100,
                         'agent1':10,  
                         'Starch':10,
                         },
                         dt=0.1,
                         episode_time=100,
                         number_of_batches=5000,
                         episodes_per_batch=int(NUM_CORES/2),
                         )                 


env.rewards={agent.name:[] for agent in env.agents}

if not os.path.exists(f"Results/{env.name}"):
	os.makedirs(f"Results/{env.name}")

for agent in env.agents:
	agent.model.solver="glpk"

for batch in range(env.number_of_batches):

	batch_obs,batch_acts, batch_log_probs, batch_rtgs=tk.rollout(env)
	for agent in env.agents:
		V, _= agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
		A_k = batch_rtgs[agent.name] - V.detach()   
		A_k = (A_k - A_k.mean()) / (A_k.std() + 1e-5) 
		if batch==0:
			rich.print("[bold yellow] Hold on, bringing the creitc network to range...[/bold yellow]")
			err=21
			while err>20:
				V, _= agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
				critic_loss = nn.MSELoss()(V, batch_rtgs[agent.name])
				agent.optimizer_value_.zero_grad()
				critic_loss.backward()
				agent.optimizer_value_.step() 
				err=critic_loss.item()
			rich.print("[bold green] Done![/bold green]")
		else: 
			
			for _ in range(agent.grad_updates):                                                      
				V, curr_log_probs = agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
				ratios = torch.exp(curr_log_probs - batch_log_probs[agent.name])
				surr1 = ratios * A_k.detach()
				surr2 = torch.clamp(ratios, 1 - agent.clip, 1 + agent.clip) * A_k
				actor_loss = (-torch.min(surr1, surr2)).mean()
				critic_loss = nn.MSELoss()(V, batch_rtgs[agent.name])
				agent.optimizer_policy_.zero_grad()
				actor_loss.backward(retain_graph=False)
				agent.optimizer_policy_.step()
				agent.optimizer_value_.zero_grad()
				critic_loss.backward()
				agent.optimizer_value_.step()                                                            
	
		if batch%500==0:
		
			with open(f"Results/{env.name}/{env.name}_{batch}.pkl", 'wb') as f:
				pickle.dump(env, f)
			with open(f"Results/{env.name}/observations_{batch}.pkl", 'wb') as f:
				pickle.dump(batch_obs,f)		
			with open(f"Results/{env.name}/actions_{batch}.pkl", 'wb') as f:
				pickle.dump(batch_acts,f)		
          		
		print(f"Batch {batch} finished:")
		for agent in env.agents:
			print(f"{agent.name} return is:  {np.mean(env.rewards[agent.name][-env.episodes_per_batch:])}")	





## Toy-NECOM-Auxotrophs

In [6]:
agent1=tk.Agent("agent1",
				model=tm.Toy_Model_NE_Aux_1,
				actor_network=tk.NN,
				critic_network=tk.NN,
				clip=0.1,
				lr_actor=0.0001,
				lr_critic=0.001,
				grad_updates=10,
				optimizer_actor=torch.optim.Adam,
				optimizer_critic=torch.optim.Adam,       
				observables=['agent1','agent2','S',"A","B"],
				actions=['A_e','B_e'],
				gamma=1,
				)
agent2=tk.Agent("agent2",
				model=tm.Toy_Model_NE_Aux_2,
				actor_network=tk.NN,
				critic_network=tk.NN,
				clip=0.1,
				lr_actor=0.0001,
				lr_critic=0.001,
				grad_updates=10,
				optimizer_actor=torch.optim.Adam,
				optimizer_critic=torch.optim.Adam,       
				observables=['agent1','agent2','S',"A","B"],
				actions=['A_e','B_e'],
				gamma=1)
agents=[agent1,agent2]

env=tk.Environment(name="Toy-NECOM-Auxotrophs",
 					agents=agents,
 					dilution_rate=0.0001,
 					extracellular_reactions=[],
 					initial_condition={"S":100,"agent1":0.1,"agent2":0.1},
 					inlet_conditions={"S":100},
 					max_c={'S':100,
 						   'agent1':10,  
 						   'agent2':10,
 						   'A':10,
 						   'B':10,},
 							dt=0.1,
 							episode_time=100,
 							number_of_batches=5000,
 							episodes_per_batch=NUM_CORES,)

env.rewards={agent.name:[] for agent in env.agents}
if not os.path.exists(f"Results/{env.name}"):
	os.makedirs(f"Results/{env.name}")
for agent in env.agents:
	agent.model.solver="glpk"
for batch in range(env.number_of_batches):
	batch_obs,batch_acts, batch_log_probs, batch_rtgs=tk.rollout(env)
	for agent in env.agents:
		V, _= agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
		A_k = batch_rtgs[agent.name] - V.detach()   
		A_k = (A_k - A_k.mean()) / (A_k.std() + 1e-5) 
		if batch==0:
			rich.print("[bold yellow] Hold on, bringing the creitc network to range...[/bold yellow]")
			err=51
			while err>50:
				V, _= agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
				critic_loss = nn.MSELoss()(V, batch_rtgs[agent.name])
				agent.optimizer_value_.zero_grad()
				critic_loss.backward()
				agent.optimizer_value_.step() 
				err=critic_loss.item()
			rich.print("[bold green] Done![/bold green]")
		else: 			
			for _ in range(agent.grad_updates):                                                      
				V, curr_log_probs = agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
				ratios = torch.exp(curr_log_probs - batch_log_probs[agent.name])
				surr1 = ratios * A_k.detach()
				surr2 = torch.clamp(ratios, 1 - agent.clip, 1 + agent.clip) * A_k
				actor_loss = (-torch.min(surr1, surr2)).mean()
				critic_loss = nn.MSELoss()(V, batch_rtgs[agent.name])
				agent.optimizer_policy_.zero_grad()
				actor_loss.backward(retain_graph=False)
				agent.optimizer_policy_.step()
				agent.optimizer_value_.zero_grad()
				critic_loss.backward()
				agent.optimizer_value_.step()                                                            	
	if batch%500==0:		
		with open(f"Results/{env.name}/{env.name}_{batch}.pkl", 'wb') as f:
			pickle.dump(env, f)
		with open(f"Results/{env.name}/observations_{batch}.pkl", 'wb') as f:
			pickle.dump(batch_obs,f)		
		with open(f"Results/{env.name}/actions_{batch}.pkl", 'wb') as f:
			pickle.dump(batch_acts,f)		
    		
	print(f"Batch {batch} finished:")
	for agent in env.agents:
		print(f"{agent.name} return is:  {np.mean(env.rewards[agent.name][-env.episodes_per_batch:])}")		

Environment Toy-NECOM-Auxotrophs created successfully!.


 Hold on, bringing the creitc network to range...

 Done!

 Hold on, bringing the creitc network to range...

 Done!

Batch 0 finished:
agent1 return is:  -693.263211649023
agent2 return is:  -494.3485401777354
Batch 1 finished:
agent1 return is:  -689.7608206478573
agent2 return is:  -489.10863895799946
Batch 2 finished:
agent1 return is:  -686.385095640583
agent2 return is:  -500.47163949786096
Batch 3 finished:
agent1 return is:  -697.5145220951706
agent2 return is:  -505.0906397532352
Batch 4 finished:
agent1 return is:  -701.8949813944643
agent2 return is:  -510.2214796378634
Batch 5 finished:
agent1 return is:  -688.8917124463186
agent2 return is:  -506.8473380565723
Batch 6 finished:
agent1 return is:  -696.5160124816357
agent2 return is:  -501.72032076966195
Batch 7 finished:
agent1 return is:  -680.2570645901875
agent2 return is:  -490.71814468345343
Batch 8 finished:
agent1 return is:  -684.3835380369874
agent2 return is:  -464.32930403313355
Batch 9 finished:
agent1 return is:  -688.5053279676099
agent2 return is:  -459.6008065688017
Batch 10 finished:
agent1 return is:  -691.6378828485514


KeyboardInterrupt: 

## Toy-NECOM-Facultative

In [8]:
agent1=tk.Agent("agent1",
				model=tm.Toy_Model_NE_Mut_1,
				actor_network=tk.NN,
				critic_network=tk.NN,
				clip=0.1,
				lr_actor=0.0001,
				lr_critic=0.001,
				grad_updates=10,
				optimizer_actor=torch.optim.Adam,
				optimizer_critic=torch.optim.Adam,       
				observables=['agent1','agent2','S',"A","B"],
				actions=['A_e','B_e'],
				gamma=1,
				)
agent2=tk.Agent("agent2",
				model=tm.Toy_Model_NE_Mut_2,
				actor_network=tk.NN,
				critic_network=tk.NN,
				clip=0.1,
				lr_actor=0.0001,
				lr_critic=0.001,
				grad_updates=10,
				optimizer_actor=torch.optim.Adam,
				optimizer_critic=torch.optim.Adam,       
				observables=['agent1','agent2','S',"A","B"],
				actions=['A_e','B_e'],
				gamma=1)
agents=[agent1,agent2]

env=tk.Environment(name="Toy-NECOM-Facultative",
 					agents=agents,
 					dilution_rate=0.0001,
 					extracellular_reactions=[],
 					initial_condition={"S":100,"agent1":0.1,"agent2":0.1,"A":0,"B":0},
 					inlet_conditions={"S":100},
 					max_c={'S':100,
 						   'agent1':10,  
 						   'agent2':10,
 						   'A':10,
 						   'B':10,},
 							dt=0.1,
 							episode_time=100,
 							number_of_batches=5000,
 							episodes_per_batch=4,)

env.rewards={agent.name:[] for agent in env.agents}
if not os.path.exists(f"Results/{env.name}"):
	os.makedirs(f"Results/{env.name}")
for agent in env.agents:
	agent.model.solver="glpk"
for batch in range(env.number_of_batches):
	batch_obs,batch_acts, batch_log_probs, batch_rtgs=tk.rollout(env)
	for agent in env.agents:
		V, _= agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
		A_k = batch_rtgs[agent.name] - V.detach()   
		A_k = (A_k - A_k.mean()) / (A_k.std() + 1e-5) 
		if batch==0:
			rich.print("[bold yellow] Hold on, bringing the creitc network to range...[/bold yellow]")
			err=1001
			while err>1000:
				V, _= agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
				critic_loss = nn.MSELoss()(V, batch_rtgs[agent.name])
				agent.optimizer_value_.zero_grad()
				critic_loss.backward()
				agent.optimizer_value_.step() 
				err=critic_loss.item()
			rich.print("[bold green] Done![/bold green]")
		else: 			
			for _ in range(agent.grad_updates):                                                      
				V, curr_log_probs = agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
				ratios = torch.exp(curr_log_probs - batch_log_probs[agent.name])
				surr1 = ratios * A_k.detach()
				surr2 = torch.clamp(ratios, 1 - agent.clip, 1 + agent.clip) * A_k
				actor_loss = (-torch.min(surr1, surr2)).mean()
				critic_loss = nn.MSELoss()(V, batch_rtgs[agent.name])
				agent.optimizer_policy_.zero_grad()
				actor_loss.backward(retain_graph=False)
				agent.optimizer_policy_.step()
				agent.optimizer_value_.zero_grad()
				critic_loss.backward()
				agent.optimizer_value_.step()                                                            	
		if batch%500==0:		
			with open(f"Results/{env.name}/{env.name}_{batch}.pkl", 'wb') as f:
				pickle.dump(env, f)
			with open(f"Results/{env.name}/observations_{batch}.pkl", 'wb') as f:
				pickle.dump(batch_obs,f)		
			with open(f"Results/{env.name}/actions_{batch}.pkl", 'wb') as f:
				pickle.dump(batch_acts,f)		
        		
		print(f"Batch {batch} finished:")
		for agent in env.agents:
			print(f"{agent.name} return is:  {np.mean(env.rewards[agent.name][-env.episodes_per_batch:])}")		

Environment Toy-NECOM-Facultative created successfully!.


 Hold on, bringing the creitc network to range...

KeyboardInterrupt: 

## Toy-Exoenzyme-Five-Agents-with-mass-transfer

In [ ]:


from mimetypes import init
from turtle import color
import Toolkit as tk
import Toy_Exoenzyme_mass_transfer as tm
import torch
import torch.functional as F
import torch.nn as nn
import numpy as np
import pandas as pd
import pickle
import time
import ray
import os
import seaborn  as sns
import matplotlib.pyplot as plt
import warnings
import json
import rich
import multiprocessing as mp
import cobra

NUM_CORES = 8
warnings.filterwarnings("ignore") 

toy_model_1=tm.ToyModel_SA_1.copy()
toy_model_2=tm.ToyModel_SA_2.copy()
toy_model_3=tm.ToyModel_SA_3.copy()
toy_model_4=tm.ToyModel_SA_4.copy()
toy_model_5=tm.ToyModel_SA_5.copy()


agent1=tk.Agent("agent1",
                model=toy_model_1,
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=4,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1','agent2','agent3','agent4','agent5' ,'Glc_1', 'Starch'],
                actions=["Amylase_1_e"],
                gamma=1,
                tau=0.1
                )

agent2=tk.Agent("agent2",
                model=toy_model_2,
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=4,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1','agent2','agent3','agent4','agent5' ,'Glc_2', 'Starch'],
                actions=["Amylase_2_e"],
                gamma=1,
                tau=0.1
                )



agent3=tk.Agent("agent3",
                model=toy_model_3,
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=4,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
				observables=['agent1','agent2','agent3','agent4','agent5' ,'Glc_3', 'Starch'],
                actions=["Amylase_3_e"],
                gamma=1,
                tau=0.1
)

agent4=tk.Agent("agent4",
                model=toy_model_4,
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=4,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
				observables=['agent1','agent2','agent3','agent4','agent5' ,'Glc_4', 'Starch'],
                actions=["Amylase_4_e"],
                gamma=1,
                tau=0.1
)

agent5=tk.Agent("agent5",
                model=toy_model_5,
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=4,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
				observables=['agent1','agent2','agent3','agent4','agent5' ,'Glc_5', 'Starch'],
                actions=["Amylase_5_e"],
                gamma=1,
                tau=0.1
)

agents=[agent1,agent2,agent3,agent4,agent5]

env=tk.Environment(name="Toy-Exoenzyme-Five-agents-mass-transfer-low",
                    agents=agents,
                    dilution_rate=0.0001,
                    initial_condition={"Glc":0,"Glc_1":20,"Glc_2":20,"Glc_3":20,"Glc_4":20,"Glc_5":20,"agent1":0.1,"agent2":0.1,'agent3':0.1,'agent4':0.1,'agent5':0.1,"Starch":10},
                    inlet_conditions={"Starch":10},
                    extracellular_reactions=[{"reaction":{
                    "Glc_1":10,
                    "Starch":-0.1,},
                    "kinetics": (tk.general_kinetic,("Glc_1","Amylase_1"))}
		            ,
		            {
                    "reaction":{
                    "Glc_2":10,
                    "Starch":-0.1,},
                    "kinetics": (tk.general_kinetic,("Glc_2","Amylase_2"))
                    ,}
		    ,
		    {
                    "reaction":{
                    "Glc_3":10,
                    "Starch":-0.1,},
                    "kinetics": (tk.general_kinetic,("Glc_3","Amylase_3"))
		    
            },
            {
                    "reaction":{
                    "Glc_4":10,
                    "Starch":-0.1,},
                    "kinetics": (tk.general_kinetic,("Glc_4","Amylase_4"))
                    
            },  
	    
            {   "reaction":{
                    "Glc_5":10,
                    "Starch":-0.1,},
                    "kinetics": (tk.general_kinetic,("Glc_5","Amylase_5"))
                    
            }  ,
            {   
                "reaction":{
                    "Glc_1":-1,
                    "Glc":1},
                "kinetics": (tk.mass_transfer,("Glc_1","Glc"))
            }
            ,
            {
                "reaction":{
                    "Glc_2":-1,
                    "Glc":1},
                "kinetics": (tk.mass_transfer,("Glc_2","Glc"))
            }
            ,
            {
                "reaction":{
                    "Glc_3":-1,
                    "Glc":1},
                "kinetics": (tk.mass_transfer,("Glc_3","Glc"))  
            },
            {
                "reaction":{
                    "Glc_4":-1,
                    "Glc":1},
                "kinetics": (tk.mass_transfer,("Glc_4","Glc"))
            }
            ,
            {
                "reaction":{ 
                    "Glc_5":-1,
                    "Glc":1},
                "kinetics": (tk.mass_transfer,("Glc_5","Glc"))
            }
					],
                    max_c={'Glc':100,
                           'agent1':10,  
                           'Starch':10,
                           },
                           dt=0.1,
                           episode_time=100,
                           number_of_batches=5000,
                           episodes_per_batch=int(NUM_CORES/2),
                           )

env.rewards={agent.name:[] for agent in env.agents}

if not os.path.exists(f"Results/{env.name}"):
	os.makedirs(f"Results/{env.name}")

for agent in env.agents:
	agent.model.solver="glpk"

for batch in range(env.number_of_batches):

	batch_obs,batch_acts, batch_log_probs, batch_rtgs=tk.rollout(env)
	for agent in env.agents:
		V, _= agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
		A_k = batch_rtgs[agent.name] - V.detach()   
		A_k = (A_k - A_k.mean()) / (A_k.std() + 1e-5) 
		if batch==0:
			rich.print("[bold yellow] Hold on, bringing the creitc network to range...[/bold yellow]")
			err=51
			while err>50:
				V, _= agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
				critic_loss = nn.MSELoss()(V, batch_rtgs[agent.name])
				agent.optimizer_value_.zero_grad()
				critic_loss.backward()
				agent.optimizer_value_.step() 
				err=critic_loss.item()
			rich.print("[bold green] Done![/bold green]")
		else: 
			
			for _ in range(agent.grad_updates):                                                      
				V, curr_log_probs = agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
				ratios = torch.exp(curr_log_probs - batch_log_probs[agent.name])
				surr1 = ratios * A_k.detach()
				surr2 = torch.clamp(ratios, 1 - agent.clip, 1 + agent.clip) * A_k
				actor_loss = (-torch.min(surr1, surr2)).mean()
				critic_loss = nn.MSELoss()(V, batch_rtgs[agent.name])
				agent.optimizer_policy_.zero_grad()
				actor_loss.backward(retain_graph=False)
				agent.optimizer_policy_.step()
				agent.optimizer_value_.zero_grad()
				critic_loss.backward()
				agent.optimizer_value_.step()                                                            
	
	if batch%500==0:
	
		with open(f"Results/{env.name}/{env.name}_{batch}.pkl", 'wb') as f:
			pickle.dump(env, f)
		with open(f"Results/{env.name}/observations_{batch}.pkl", 'wb') as f:
			pickle.dump(batch_obs,f)		
		with open(f"Results/{env.name}/actions_{batch}.pkl", 'wb') as f:
			pickle.dump(batch_acts,f)		
      		
	print(f"Batch {batch} finished:")
	for agent in env.agents:
		print(f"{agent.name} return is:  {np.mean(env.rewards[agent.name][-env.episodes_per_batch:])}")	

## IJO1366-Tyr-Phe-Auxotrophs

In [ ]:

from mimetypes import init
from turtle import color
import Toolkit as tk
import Toy_Exoenzyme_mass_transfer as tm
import torch
import torch.functional as F
import torch.nn as nn
import numpy as np
import pandas as pd
import pickle
import time
import ray
import os
import seaborn  as sns
import matplotlib.pyplot as plt
import warnings
import json
import rich
import multiprocessing as mp
import cobra
from itertools import combinations

NUM_CORES = 8
warnings.filterwarnings("ignore") 

model_base = cobra.io.read_sbml_model("iJO1366.xml")
medium = model_base.medium.copy()
test_model = model_base.copy()

knockouts_gene_names = [
    "tyrA",
    "pheA",
]

exchange_reactions = {
    "tyrA": "EX_tyr__L_e",
    "pheA": "EX_phe__L_e",
    # "argA": "EX_arg__L_e",
    # "hisB": "EX_his__L_e",
    # "leuB": "EX_leu__L_e",
    # "cysE": "EX_cys__L_e",
    # "glyA": "EX_gly_e",
    # "serA": "EX_ser__L_e",
    # "thrC": "EX_thr__L_e",
    # "trpC": "EX_trp__L_e",
    # "ilvA": "EX_ile__L_e",
    # "lysA": "EX_lys__L_e",
    # "metA": "EX_met__L_e",
    # "proA": "EX_pro__L_e",
}

exchange_species = {}
exchange_mets = {}
for i in exchange_reactions.items():
    exchange_mets[i[0]] = list(test_model.reactions.get_by_id(i[1]).metabolites.keys())[
        0
    ].id

gene_ids = {}
for ko_gene in knockouts_gene_names:
    for gene in test_model.genes:
        if gene.name == ko_gene:
            gene_ids[ko_gene] = gene.id

            


knockouts = set()
for i in combinations(knockouts_gene_names, 2):
    if set(i) not in knockouts:
        knockouts.add(frozenset(i))

unique_knockouts = [tuple(i) for i in knockouts]

ic={
    key.lstrip("EX_"):3 for key,val in model_base.medium.items() 
}

ic['glc__D_e']=500
ic['agent1']=0.1
ic['agent2']=0.1
for ko in [("tyrA","pheA")]:
    model1 = model_base.copy()
    model2 = model_base.copy()
    model1.remove_reactions([model1.reactions.get_by_id('PPND')]) ## Tyrosine Mutant
    model2.remove_reactions([model2.reactions.get_by_id('PPNDH')]) ## Phenylalanine Mutant
    model1.exchange_reactions = tuple([model1.reactions.index(i) for i in model1.exchanges])
    model2.exchange_reactions = tuple([model2.reactions.index(i) for i in model2.exchanges])
    model1.biomass_ind=model1.reactions.index("BIOMASS_Ec_iJO1366_core_53p95M")
    model2.biomass_ind=model2.reactions.index("BIOMASS_Ec_iJO1366_core_53p95M")
    model1.solver = "gurobi"
    model2.solver = "gurobi"
    if model1.optimize().objective_value != 0 or model2.optimize().objective_value != 0:
        rich.print(f"[yellow]Skipping {ko} because at least one organism can grow without auxotrophy")
        continue
    else:
        rich.print(f"[green]Non of the KOs can grow without auxotrophy: Running {ko}")
    ko_name = ko[0] + "_" + ko[1]
    agent1 = tk.Agent(
        "agent1",
        model=model1,
        actor_network=tk.NN,
        critic_network=tk.NN,
        clip=0.1,
        lr_actor=0.0001,
        lr_critic=0.001,
        actor_var=0.05,
        grad_updates=1,
        optimizer_actor=torch.optim.Adam,
        optimizer_critic=torch.optim.Adam,
        observables=[
            "agent1",
            "agent2",
            "glc__D_e",
            *[i.replace("EX_", "") for i in exchange_reactions.values()]
        ],
        actions=[i for i in exchange_reactions.values()],
        gamma=1,
    )
    agent2 = tk.Agent(
        "agent2",
        model=model2,
        actor_network=tk.NN,
        critic_network=tk.NN,
        clip=0.1,
        lr_actor=0.0001,
        lr_critic=0.001,
        grad_updates=1,
        actor_var=0.05,
        optimizer_actor=torch.optim.Adam,
        optimizer_critic=torch.optim.Adam,
        observables=[
            "agent1",
            "agent2",
            "glc__D_e",
            *[i.replace("EX_", "") for i in exchange_reactions.values()]
        ],
        actions=[i for i in exchange_reactions.values()],
        gamma=1,
    )
    constants=list(ic.keys())
    constants.remove("agent1")
    constants.remove("agent2")
    constants.remove("glc__D_e")

    env = tk.Environment(
        "IJO1366-Tyr-Phe-Auxotrophs" ,
        agents=[agent1, agent2],
        dilution_rate=0,
        extracellular_reactions=[],
        initial_condition=ic,
        inlet_conditions={},
        max_c={},
        dt=0.2,
        episode_time=20,  ##TOBECHANGED
        number_of_batches=10000,  ##TOBECHANGED
        episodes_per_batch=4,
        constant=constants,
    )

    env.rewards = {agent.name: [] for agent in env.agents}

    if not os.path.exists(f"Results/{env.name}"):
        os.makedirs(f"Results/{env.name}")
### The next block will train the actor network to output -1 for all actions, so that 
### the agents start like FBA

    for batch in range(env.number_of_batches):
        batch_obs,batch_acts, batch_log_probs, batch_rtgs=tk.rollout(env)
        for agent in env.agents:
            V, _= agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
            A_k = batch_rtgs[agent.name] - V.detach()   
            A_k = (A_k - A_k.mean()) / (A_k.std() + 1e-5) 
            if batch==0:
                rich.print("[bold yellow] Hold on, bringing the networks to range...[/bold yellow]")
                err=101
                while err>100:
                    V, _= agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
                    critic_loss = nn.MSELoss()(V, batch_rtgs[agent.name])
                    agent.optimizer_value_.zero_grad()
                    critic_loss.backward()
                    agent.optimizer_value_.step() 
                    err=critic_loss.item()
                rich.print("[bold green] Done![/bold green]")
            else: 			
                for _ in range(agent.grad_updates):                                                      
                    V, curr_log_probs = agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
                    ratios = torch.exp(curr_log_probs - batch_log_probs[agent.name])
                    surr1 = ratios * A_k.detach()
                    surr2 = torch.clamp(ratios, 1 - agent.clip, 1 + agent.clip) * A_k
                    actor_loss = (-torch.min(surr1, surr2)).mean()
                    critic_loss = nn.MSELoss()(V, batch_rtgs[agent.name])
                    agent.optimizer_policy_.zero_grad()
                    actor_loss.backward(retain_graph=False)
                    agent.optimizer_policy_.step()
                    agent.optimizer_value_.zero_grad()
                    critic_loss.backward()
                    agent.optimizer_value_.step()   

        if batch%500==0:
            with open(f"Results/{env.name}/{env.name}_{batch}.pkl", 'wb') as f:
                pickle.dump(env, f)
            with open(f"Results/{env.name}/observations_{batch}.pkl", 'wb') as f:
                pickle.dump(batch_obs,f)		
            with open(f"Results/{env.name}/actions_{batch}.pkl", 'wb') as f:
                pickle.dump(batch_acts,f)		
        print(f"Batch {batch} finished:")
        for agent in env.agents:
        	print(f"{agent.name} return is:  {np.mean(env.rewards[agent.name][-env.episodes_per_batch:])}")	